In [53]:
# Code to create a model used to predict whether the individual patient has an allergy
# Use different notebook to load the model and return a prediction

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow
import keras

import numpy as np
np.random.seed(123)

In [54]:
import matplotlib
from keras.models import * 
from keras.layers import *
from keras.optimizers import RMSprop
import pandas as pd

# Import both datasets, change to local path when running
patients = pd.read_excel(r"C:\Users\ncros\OneDrive\Desktop\492\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level2_AI_Patient Traits")

allergies = pd.read_excel(r"C:\Users\ncros\OneDrive\Desktop\492\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level1_Patient Allergens")

In [55]:
# Confirm that patient sheet imported correctly
# Comment below line before committing
# patients['SkinConditions'].head()

In [56]:
# Confirm allergy sheet imported correctly
# Comment line before committing
# allergies.head()

In [57]:
# Merge columns by ID if needed
patientAllergies = patients.merge(allergies, on = "SFM Id")
# Comment line before committing
# patientAllergies.head()

In [58]:
# Drop ID and location columns from dataframe
patientsTrimmed = patients.drop(['SFM Id', 'City', 'State', 'Country'], axis = 1)

In [59]:
# One-hot encode Gender column
patientsTrimmed['Gender'] = pd.Categorical(patientsTrimmed['Gender'].str.strip())
gender_onehot = pd.get_dummies(patientsTrimmed['Gender'], prefix = "Gender",
                                    prefix_sep = "-", dtype = int)
patientsTrimmed = patientsTrimmed.drop('Gender', axis = 1)
patientsTrimmed = patientsTrimmed.join(gender_onehot)

In [60]:
# One-hot encode SkinTone column
patientsTrimmed['SkinTone'] = pd.Categorical(patientsTrimmed['SkinTone'].str.strip())
skintone_onehot = pd.get_dummies(patientsTrimmed['SkinTone'], prefix = "SkinTone",
                                    prefix_sep = "-", dtype = int)
patientsTrimmed = patientsTrimmed.drop('SkinTone', axis = 1)
patientsTrimmed = patientsTrimmed.join(skintone_onehot)

In [61]:
# One-hot encode FitzPatrickSkinPhotoType column
patientsTrimmed['FitzpatrickSkinPhotoType'] = pd.Categorical(patientsTrimmed['FitzpatrickSkinPhotoType'].str.strip())
# Dropping first here since it is a blank variable in the column
fitzpatrick_onehot = pd.get_dummies(patientsTrimmed['FitzpatrickSkinPhotoType'], prefix = "Fitzpatrick",
                                    prefix_sep = "-", drop_first = True, dtype = int)
patientsTrimmed = patientsTrimmed.drop('FitzpatrickSkinPhotoType', axis = 1)
patientsTrimmed = patientsTrimmed.join(fitzpatrick_onehot)

In [62]:
# Switching to TextVectorization (Tokenizer is deprecated)
from keras.layers import TextVectorization
# Replace commas with whitespace
patientsTrimmed['SkinConditions'] = patientsTrimmed['SkinConditions'].str.replace(',', ' ')
# Set the max length based on whitespace characters
max_len = patientsTrimmed['SkinConditions'].str.count(' ').max()
# Create TextVectorization object, separating on whitespace and using the max_len from earlier
vectorizer = TextVectorization(split = 'whitespace', output_sequence_length = max_len)

In [63]:
# Adapt using the column we want to convert
vectorizer.adapt(patientsTrimmed['SkinConditions'].values)
# Reset the column after converting values to vector and placing in array
skinConditions = vectorizer(patientsTrimmed['SkinConditions']).numpy()
patientsTrimmed = patientsTrimmed.drop('SkinConditions', axis = 1)

In [64]:
patientsArr = patientsTrimmed.values
input_data = np.concatenate((patientsArr, skinConditions), axis = 1)

In [65]:
''' Old code for SkinConditions conversion - leaving in case we need later
# Code to preprocess the SkinConditions column, tokenizing should be more recognizable to keras than regular text
## Delete if this idea does not work
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Flatten entries into actual lists
patientsTrimmed['SkinConditions'] = patientsTrimmed['SkinConditions'].apply(lambda x: ' '.join(x))
# Tokenize the characters of the columns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patientsTrimmed['SkinConditions'])
# Create and pad sequences of the tokens
patientsTrimmed['SkinConditions'] = tokenizer.texts_to_sequences(patientsTrimmed['SkinConditions'])
max_length = max(len(seq) for seq in patientsTrimmed['SkinConditions'])
testVar = pad_sequences(patientsTrimmed['SkinConditions'], maxlen = max_length, padding = "post")
patientsTrimmed['SkinConditions'] = testVar.tolist()
'''

' Old code for SkinConditions conversion - leaving in case we need later\n# Code to preprocess the SkinConditions column, tokenizing should be more recognizable to keras than regular text\n## Delete if this idea does not work\nfrom keras.preprocessing.text import Tokenizer\nfrom keras.preprocessing.sequence import pad_sequences\n\n# Flatten entries into actual lists\npatientsTrimmed[\'SkinConditions\'] = patientsTrimmed[\'SkinConditions\'].apply(lambda x: \' \'.join(x))\n# Tokenize the characters of the columns\ntokenizer = Tokenizer()\ntokenizer.fit_on_texts(patientsTrimmed[\'SkinConditions\'])\n# Create and pad sequences of the tokens\npatientsTrimmed[\'SkinConditions\'] = tokenizer.texts_to_sequences(patientsTrimmed[\'SkinConditions\'])\nmax_length = max(len(seq) for seq in patientsTrimmed[\'SkinConditions\'])\ntestVar = pad_sequences(patientsTrimmed[\'SkinConditions\'], maxlen = max_length, padding = "post")\npatientsTrimmed[\'SkinConditions\'] = testVar.tolist()\n'

In [66]:
# Drop ID column for preprocessing - ID should have no effect on prediction
allergiesNoId = allergies.drop('SFM Id', axis = 1)
# Remove all non-digit characters, then replace empty cells with NaN
allergiesNoId = allergiesNoId.replace(r'\D+', '', regex = True).replace('', np.nan)
# Set all NaN cells to 0
allergiesNoId = allergiesNoId.fillna(0)
# Convert entire dataframe to integer
allergiesNoId = allergiesNoId.astype(int)

In [67]:
allergiesNoId['AllergiesList'] = allergiesNoId.astype(str).apply(' '.join, axis=1)
allergiesNoId['AllergiesList'] = allergiesNoId['AllergiesList'].str.split()

In [68]:
from sklearn.preprocessing import MultiLabelBinarizer
# Create MultiLabelBinarizer object
mlb = MultiLabelBinarizer()

allergiesArr = np.array(allergiesNoId['AllergiesList'])
# Multi-hot encode data
allergiesArray = mlb.fit_transform(allergiesArr)

In [69]:
allergiesNew = mlb.inverse_transform(allergiesArray)

In [70]:
allergiesDF = pd.DataFrame(mlb.transform(allergiesArr), columns = mlb.classes_)
allergiesDF.dtypes

0         int32
100612    int32
100613    int32
100702    int32
100857    int32
          ...  
9804      int32
98288     int32
99100     int32
9926      int32
99356     int32
Length: 731, dtype: object

In [80]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix
from keras.layers import Average

cv = KFold(n_splits=5, shuffle=True, random_state=7869)

#weight_array = compute_class_weight(class_weight = 'balanced', 
                                    #classes = np.unique(np.argmax(allergiesArray, axis=1)), 
                                    #y = np.argmax(allergiesArray, axis=1))
#print(weight_array)
#weight_dict = dict(zip(np.unique(np.argmax(allergiesArray, axis=1)), weight_array))
#print(weight_dict)

fold_count = 1
# Train the model for each split
# Define the model inside the for loop
for train, test in cv.split(input_data,allergiesArray):

    n_classes = 731

    # Input layer
    input_shape = (40,)
    inputs = keras.Input(input_shape)

    # Hidden layers
    x = Dense(128, activation='sigmoid')(inputs)
    x = Dense(512, activation = 'sigmoid')(x)
    x = Dense(2048, activation = 'sigmoid')(x)

    # Output layer - use multilabel classification
    predictions = Dense(n_classes, activation='sigmoid')(x)
    
    model = keras.Model(inputs = inputs, outputs = predictions)
    model.compile(loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha = 0.05, gamma = 12), optimizer=keras.optimizers.Adam(learning_rate=0.1))
    
    print("Running fold #: ", fold_count)

    fold_train_x =input_data[train]
    
    history = model.fit(
    fold_train_x, allergiesArray[train],
    epochs = 20, 
    verbose = 1, 
    validation_split=0.2
    )

    fold_test_x = input_data[test]
    y_true = allergiesArray[test]
    
    # pred_y = model.predict_classes(fold_test_x, verbose = 1)
    probs = model.predict(fold_test_x, verbose = 1)
    

    # AUC (Area Under Curve): how well the model can classify into the classes (high numbers better)
    auc = keras.metrics.AUC(multi_label = True, num_labels = 731, from_logits = False)
    # Precision: how well model predicts target class (high numbers better)\
    prec = keras.metrics.Precision()
    # Recall: how many objects model can find (high numbers better)
    rec = keras.metrics.Recall()
    # F1Score: mean between precision and recall
    f1 = keras.metrics.F1Score(average = 'weighted')

    threshold = 0.35
    probs = (probs > threshold).astype(int)
    # print("Predicted y values: ", pred_y[0])
    print("X values for test: ", fold_test_x[0])
    yList = mlb.inverse_transform(y_true)
    print("Actual y values as list: ", yList[0])
    probsList = mlb.inverse_transform(probs)
    print("Predicted values as list: ", probsList[0])
    print(probs[0])
    
    auc.update_state(y_true, probs)
    print("Area under the curve: ", auc.result().numpy())
    prec.update_state(y_true, probs)
    print("Precision: ",prec.result().numpy())
    rec.update_state(y_true, probs)
    print("Recall: ",rec.result().numpy())
    f1.update_state(y_true, probs)
    print("Precision/Recall avg: ",f1.result().numpy())
    
    matrix = multilabel_confusion_matrix(y_true, probs)
    print(matrix)
    
    # print("Accuracy score sklearn: ", accuracy_score(allergiesArray[test], probs))
    accuracy = (allergiesArray[test] == probs).all(axis=(0,1)).mean()
    # print("Accuracy score equation: ", accuracy)

    fold_count = fold_count + 1

Running fold #:  1
Epoch 1/20
198/198 [==============================] - 10s 36ms/step - loss: 0.1357 - val_loss: 0.1584
Epoch 2/20
198/198 [==============================] - 7s 35ms/step - loss: 0.1275 - val_loss: 0.1230
Epoch 3/20
198/198 [==============================] - 7s 35ms/step - loss: 0.1333 - val_loss: 0.1180
Epoch 4/20
198/198 [==============================] - 7s 35ms/step - loss: 0.1391 - val_loss: 0.1376
Epoch 5/20
198/198 [==============================] - 7s 35ms/step - loss: 0.1436 - val_loss: 0.1460
Epoch 6/20
198/198 [==============================] - 7s 35ms/step - loss: 0.1572 - val_loss: 0.1523
Epoch 7/20
198/198 [==============================] - 7s 35ms/step - loss: 0.1591 - val_loss: 0.1552
Epoch 8/20
198/198 [==============================] - 7s 35ms/step - loss: 0.1786 - val_loss: 0.1571
Epoch 9/20
198/198 [==============================] - 7s 35ms/step - loss: 0.1697 - val_loss: 0.1496
Epoch 10/20
198/198 [==============================] - 7s 34ms/step - l

Area under the curve:  0.3508892
Precision:  0.10402237
Recall:  0.3871509
Precision/Recall avg:  0.19630706
[[[   0    0]
  [   0 1980]]

 [[1978    0]
  [   2    0]]

 [[1977    0]
  [   3    0]]

 ...

 [[1979    0]
  [   1    0]]

 [[1979    0]
  [   1    0]]

 [[1979    0]
  [   1    0]]]
Running fold #:  3
Epoch 1/20
198/198 [==============================] - 10s 33ms/step - loss: 0.1440 - val_loss: 0.1223
Epoch 2/20
198/198 [==============================] - 7s 34ms/step - loss: 0.1328 - val_loss: 0.1408
Epoch 3/20
198/198 [==============================] - 6s 32ms/step - loss: 0.1416 - val_loss: 0.1416
Epoch 4/20
198/198 [==============================] - 6s 33ms/step - loss: 0.1571 - val_loss: 0.1597
Epoch 5/20
198/198 [==============================] - 6s 32ms/step - loss: 0.1703 - val_loss: 0.1745
Epoch 6/20
198/198 [==============================] - 6s 32ms/step - loss: 0.1819 - val_loss: 0.1932
Epoch 7/20
198/198 [==============================] - 7s 35ms/step - loss: 0.18

Area under the curve:  0.34336525
Precision:  0.10836281
Recall:  0.2985937
Precision/Recall avg:  0.17213097
[[[   0    0]
  [   0 1979]]

 [[1979    0]
  [   0    0]]

 [[1977    0]
  [   2    0]]

 ...

 [[1976    0]
  [   3    0]]

 [[1979    0]
  [   0    0]]

 [[1979    0]
  [   0    0]]]
Running fold #:  5
Epoch 1/20
198/198 [==============================] - 9s 34ms/step - loss: 0.1361 - val_loss: 0.1158
Epoch 2/20
198/198 [==============================] - 6s 31ms/step - loss: 0.1284 - val_loss: 0.1354
Epoch 3/20
198/198 [==============================] - 6s 31ms/step - loss: 0.1307 - val_loss: 0.1370
Epoch 4/20
198/198 [==============================] - 6s 31ms/step - loss: 0.1502 - val_loss: 0.1535
Epoch 5/20
198/198 [==============================] - 6s 31ms/step - loss: 0.1500 - val_loss: 0.1674
Epoch 6/20
198/198 [==============================] - 6s 31ms/step - loss: 0.1554 - val_loss: 0.1417
Epoch 7/20
198/198 [==============================] - 6s 32ms/step - loss: 0.15

In [ ]:
# TODO: Save Keras model as separate file